# Create Fact Table

**Reading Silver Data**

In [ ]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@cardatalake00.dfs.core.windows.net/carsales`")

**Reading all dim tables**

In [ ]:
df_dealer = spark.sql("SELECT * FROM cars_catalog.gold.dim_dealer")

df_branch = spark.sql("SELECT * FROM cars_catalog.gold.dim_branch")

df_date = spark.sql("SELECT * FROM cars_catalog.gold.dim_date")

df_model = spark.sql("SELECT * FROM cars_catalog.gold.dim_model")

**Bringing keys to the fact table**

In [ ]:
df_fact = df_silver.join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, how='left')\
    .join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, how='left')\
        .join(df_date, df_silver.Date_ID == df_date.Date_ID, how='left')\
            .join(df_model, df_silver.Model_ID == df_model.Model_ID, how='left')\
                .select(df_branch.dim_branch_key, df_dealer.dim_dealer_key, df_model.dim_model_key, df_date.dim_date_key, df_silver.Revenue, df_silver.Units_Sold, df_silver.RevPerUnit)

In [ ]:
df_fact.display()

###  Writing Fact Table

In [ ]:
from delta.tables import DeltaTable

In [ ]:
if spark.catalog.tableExists('cars_catalog.gold.fact_sales'):
    deltaTable=DeltaTable.forName(spark, 'cars_catalog.gold.fact_sales')
    deltaTable.alias('trg').merge(df_fact.alias('src'), 'trg.dim_branch_key=src.dim_branch_key and trg.dim_dealer_key=src.dim_dealer_key and trg.dim_model_key=src.dim_model_key and trg.dim_date_key=src.dim_date_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

else:
    df_fact.write.format('delta')\
        .mode('overwrite')\
            .option('path', 'abfss://gold@cardatalake00.dfs.core.windows.net/fact_sales')\
                .saveAsTable('cars_catalog.gold.fact_sales')

In [ ]:
%sql
select * from cars_catalog.gold.fact_sales